In this tutorial, we'll use a pretrained transformer model to perform question answering (QA). The model will be given a context (a passage of text) and a question, and it will try to find the most relevant answer within the context.

# Import the necessary libraries:

In [7]:
from transformers import pipeline


# Create a question-answering pipeline:
We'll initialize a pipeline for question-answering using a pre-trained model.

In [2]:
question_answerer = pipeline("question-answering", model="distilbert-base-cased-distilled-squad")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


# Provide context and ask a question:
You’ll need a context paragraph (the passage in which the model should search for answers) and a specific question.

In [3]:
context = """
Transformers are a type of machine learning model introduced in 2017. They use self-attention mechanisms to
process input data. Since their introduction, they have achieved state-of-the-art performance in various natural
language processing tasks like machine translation, text summarization, and question answering. The transformer
architecture led to the creation of models like BERT, GPT, and others.
"""

question = "What tasks are transformers used for?"


# Get the answer:
Use the question_answerer() function to get the answer based on the context and question.

**Parameters:**

*   context: The passage where the model will search for the answer.
*   question: The question the model will try to answer based on the context.

In [4]:
answer = question_answerer(question=question, context=context)
print("Answer:", answer['answer'])

Answer: machine translation, text summarization, and question answering



# Fine-Tuning a Pretrained Model for Question Answering

In this section, we'll demonstrate how to fine-tune a pretrained model like `distilbert-base-cased-distilled-squad` for question-answering tasks on a custom dataset.

Fine-tuning the model on your own data helps it adapt to domain-specific language and questions.


In [2]:
! pip install datasets # if necessary

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.3/474.3 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.4 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


In [3]:
# Step 1: Load a custom question-answering dataset
# We'll use the SQuAD dataset here as an example. Replace this with your own dataset if needed.
from datasets import load_dataset

dataset = load_dataset('squad', split='train[:1%]')

# Step 2: Load the pretrained model and tokenizer
from transformers import DistilBertTokenizer, DistilBertForQuestionAnswering

tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-cased')
model = DistilBertForQuestionAnswering.from_pretrained('distilbert-base-cased')

# Step 3: Preprocess the dataset for fine-tuning
def preprocess_data(examples):
    inputs = tokenizer(
        examples['context'],
        examples['question'],
        truncation=True,
        padding='max_length',
        max_length=384
    )

    # Tokenize the answer separately
    answers = examples['answers']
    start_positions = []
    end_positions = []

    for i, answer in enumerate(answers):
        answer_text = answer['text'][0]
        start_char = answer['answer_start'][0]

        # Tokenize the context
        context = examples['context'][i]
        tokenized_context = tokenizer(context, truncation=True, padding='max_length', max_length=384)

        # Tokenize the answer
        tokenized_answer = tokenizer(answer_text, truncation=True, padding='max_length', max_length=384)

        # Find the token indices corresponding to the start and end of the answer
        start_pos = None
        end_pos = None

        # Loop through the tokenized context and look for the answer tokens
        for idx in range(len(tokenized_context['input_ids']) - len(tokenized_answer['input_ids']) + 1):
            if tokenized_context['input_ids'][idx:idx + len(tokenized_answer['input_ids'])] == tokenized_answer['input_ids']:
                start_pos = idx
                end_pos = idx + len(tokenized_answer['input_ids']) - 1
                break

        if start_pos is None or end_pos is None:
            start_pos = 0
            end_pos = 0

        start_positions.append(start_pos)
        end_positions.append(end_pos)

    inputs.update({
        'start_positions': start_positions,
        'end_positions': end_positions
    })

    return inputs



train_data = dataset.map(preprocess_data, batched=True, remove_columns=['id', 'title', 'context', 'question', 'answers'])

# Step 4: Define training arguments and initialize Trainer
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',
evaluation_strategy="no",  # Disable evaluation
    learning_rate=3e-5,
    per_device_train_batch_size=8,
    num_train_epochs=1
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data
)

# Step 5: Fine-tune the model
trainer.train()


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.62k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/1.82M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

...

Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/876 [00:00<?, ? examples/s]

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

Step,Training Loss


TrainOutput(global_step=110, training_loss=0.590543642911044, metrics={'train_runtime': 35.3129, 'train_samples_per_second': 24.807, 'train_steps_per_second': 3.115, 'total_flos': 85839086721024.0, 'train_loss': 0.590543642911044, 'epoch': 1.0})

# Now check model's performance after fine-tuning

In [8]:
model_path = '/content/results/checkpoint-110'
model_fine_tuned = DistilBertForQuestionAnswering.from_pretrained(model_path)
tokenizer_fine_tuned = DistilBertTokenizer.from_pretrained('distilbert-base-cased')

question_answerer_fine_tuned = pipeline("question-answering", model=model_fine_tuned, tokenizer=tokenizer_fine_tuned)


new_context = "This is a new context for testing the fine-tuned model."
new_question = "What is the main topic of this context?"
answer_fine_tuned = question_answerer_fine_tuned(question=new_question, context=new_context)
print("Fine-tuned model answer:", answer_fine_tuned['answer'])


Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Fine-tuned model answer: model.


# Exercise Questions:

*   Change the context to a different passage (such as one from a news article or a technical document). Does the model still provide accurate answers?
*   Modify the script to allow the user to ask multiple questions about the same context without restarting the program. What changes did you make to achieve this?
*   Experiment with different pretrained QA models from Hugging Face (e.g., bert-large-uncased-whole-word-masking-finetuned-squad). How does the performance change? Which model gives the best results in your experiments?





In [10]:
context = """
In recent years, artificial intelligence (AI) has made significant advancements in various fields. Machine learning, a subset of AI, has been widely used in image recognition, natural language processing, and recommendation systems. Deep learning, a more advanced form of machine learning, has achieved remarkable results in tasks such as image classification, speech recognition, and natural language understanding.
"""

question = "What are some applications of deep learning in AI?"

answer = question_answerer_fine_tuned(question=question, context=context)
print("Answer:", answer['answer'])

Answer: understanding.


In [14]:
context = input("Please enter the context: ")

while True:
    question = input("Please enter your question (or type 'exit' to stop): ")
    if question.lower() == 'exit':
        break
    answer = question_answerer(context, question)
    print(f"\nAnswer: {answer}")

Please enter the context: In recent years, artificial intelligence (AI) has made significant advancements in various fields. Machine learning, a subset of AI, has been widely used in image recognition, natural language processing, and recommendation systems. Deep learning, a more advanced form of machine learning, has achieved remarkable results in tasks such as image classification, speech recognition, and natural language understanding.
Please enter your question (or type 'exit' to stop): exit
